# House and Appartement pricing in Haute_Garonne (31)

The goal is to fecth data from data.gouv.fr of the house sell in the department 31 for finding some patterns and deploy a model to predict the value of an house or appartment.  
The first Notebook is about the creation of clean data.
* Source : https://files.data.gouv.fr/geo-dvf/latest/csv/  
---
  
The notebook is separated like that : \
1 - Necessary importation  
2 - Recuperation of all the datas (.csv) from 2021 to 2023 and combined in one dataframe    
3 - Simplification and cleaning of the datas (dropped unnecessary columns, filled missing datas, convert data for ML)  
4 - Saving the new DataFrame for futur analysis
  
---

## Importation

In [4]:
import os
import uuid
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import scipy.stats as stats

## Data collection

### Fetch the datas, downloading and stored in one folder

In [12]:
def get_csv(url,download_folder) : 
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    csv_links = soup.find_all('a', href=True)
    csv_links = [link for link in csv_links if link['href'].endswith('.csv')]
    
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
              
    for link in csv_links:
        file_url = urljoin(url, link['href'])
        original_file_name = os.path.basename(link['href'])
        unique_file_name = f"{uuid.uuid4()}_{original_file_name}"
        file_path = os.path.join(download_folder, unique_file_name)
        
        try :    
            file_response = requests.get(file_url)
            file_response.raise_for_status()
    
            with open(file_path, 'wb') as file:
                file.write(file_response.content)
        except requests.exceptions.RequestException as e:
            print(f'Error downloading {unique_file_name}:{e}')
    
    print(f'Download completed for URL:{url}')   

In [6]:
# Folder location
download_folder = r'C:\Users\melan\Desktop\DeveloppementInfo\Portfolio\ProjectsData\House_Price_Toulouse\CSV'

In [8]:
# urls 
urls = ['https://files.data.gouv.fr/geo-dvf/latest/csv/2023/communes/31/',
        'https://files.data.gouv.fr/geo-dvf/latest/csv/2022/communes/31/',
        'https://files.data.gouv.fr/geo-dvf/latest/csv/2021/communes/31/']

In [14]:
for url in urls :
    get_csv(url, download_folder)

Download completed for URL:https://files.data.gouv.fr/geo-dvf/latest/csv/2023/communes/31/
Download completed for URL:https://files.data.gouv.fr/geo-dvf/latest/csv/2022/communes/31/
Download completed for URL:https://files.data.gouv.fr/geo-dvf/latest/csv/2021/communes/31/


### Concatenation of all the data in one csv file

In [18]:
dfs = []
for file_name in os.listdir(download_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(download_folder, file_name)
        # Lire chaque fichier CSV et l'ajouter à la liste des DataFrames
        df = pd.read_csv(file_path)
        dfs.append(df)
        
concatenated_df = pd.concat(dfs, ignore_index=True)
output_file = os.path.join(download_folder, 'concatenated.csv')
concatenated_df.to_csv(output_file, index=False)

print(f'All CSV files have been concatenated into : {output_file}')

All CSV files have been concatenated into : C:\Users\melan\Desktop\DeveloppementInfo\Portfolio\ProjectsData\House_Price_Toulouse\CSV\concatenated.csv


## Cleaning

### Reading CSV concatenated

In [20]:
df = pd.read_csv(os.path.join(download_folder, 'concatenated.csv'))

In [22]:
df.head(3)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2022-478855,2022-01-25,1,Vente,277000.0,NaN,NaN,LA BARTHE,B001,31800.0,...,NaN,NaN,NaN,T,terres,NaN,NaN,8359.0,0.666552,43.163195
1,2022-478855,2022-01-25,1,Vente,277000.0,NaN,NaN,LA BARTHE,B001,31800.0,...,NaN,NaN,NaN,T,terres,NaN,NaN,4101.0,0.666552,43.163195
2,2022-478855,2022-01-25,1,Vente,277000.0,NaN,NaN,LA BARTHE,B001,31800.0,...,NaN,NaN,NaN,T,terres,NaN,NaN,10455.0,0.667580,43.162416


In [24]:
len(df)

267714

### Cleaning the number of lines

I am only interested in houses, apartments which are sold "normally" (not at auction, or "sold as is", ...).

In [26]:
df = df[df.type_local=="Maison"].reset_index()

In [28]:
df = df[df.nature_mutation=='Vente']

In [30]:
df = df.drop_duplicates(subset=['id_mutation'])

In [32]:
len(df)

31303

### Cleaning features

In [34]:
df = df[['valeur_fonciere','code_postal','surface_reelle_bati','nombre_pieces_principales','nature_culture','surface_terrain']]

In [36]:
df = df.reset_index(drop=True)

In [38]:
df

,valeur_fonciere,code_postal,surface_reelle_bati,nombre_pieces_principales,nature_culture,surface_terrain
0,277000.0,31800.0,180.0,5.0,sols,2248.0
1,NaN,31800.0,96.0,4.0,terrains d'agrément,750.0
2,220900.0,31800.0,129.0,4.0,sols,500.0
3,154600.0,31510.0,60.0,3.0,sols,555.0
4,18000.0,31510.0,20.0,0.0,sols,40.0
...,...,...,...,...,...,...
31298,249000.0,31390.0,87.0,4.0,sols,804.0
31299,230000.0,31390.0,100.0,5.0,sols,2937.0
31300,338000.0,31390.0,108.0,4.0,sols,705.0
31301,216000.0,31390.0,129.0,6.0,sols,80.0


### Missing data

In [40]:
def percent_missing(df):
    per_nan = 100*df.isnull().sum() /len(df)
    per_nan = per_nan[per_nan >0].sort_values()
    
    return per_nan

In [42]:
per_nan = percent_missing(df)
per_nan

surface_reelle_bati          0.003195
nombre_pieces_principales    0.003195
valeur_fonciere              0.076670
nature_culture               7.817142
surface_terrain              7.817142
dtype: float64

In [44]:
df = df.dropna(subset=['valeur_fonciere','nombre_pieces_principales','surface_reelle_bati'])

In [ ]:
df['nature_culture'] = df['nature_culture'].fillna('None')
df['surface_terrain'] = df['surface_terrain'].fillna(0)

In [48]:
per_nan = percent_missing(df)
per_nan

Series([], dtype: float64)

### Exterior

In [50]:
df['nature_culture'].value_counts()

nature_culture
sols                   25685
None                    2445
terrains d'agrément     1668
terres                   532
terrains a bâtir         352
jardins                  286
prés                     190
landes                    59
taillis simples           36
taillis sous futaie       14
vignes                     3
eaux                       3
vergers                    2
chemin de fer              1
peupleraies                1
futaies feuillues          1
Name: count, dtype: int64

In [ ]:
df['exterieur'] = df['nature_culture'].apply(lambda x: 0 if x== 'None' else 1)

In [54]:
df = df.drop(['nature_culture'],axis=1)

In [56]:
df

,valeur_fonciere,code_postal,surface_reelle_bati,nombre_pieces_principales,surface_terrain,exterieur
0,277000.0,31800.0,180.0,5.0,2248.0,1
2,220900.0,31800.0,129.0,4.0,500.0,1
3,154600.0,31510.0,60.0,3.0,555.0,1
4,18000.0,31510.0,20.0,0.0,40.0,1
5,248000.0,31190.0,92.0,4.0,528.0,1
...,...,...,...,...,...,...
31298,249000.0,31390.0,87.0,4.0,804.0,1
31299,230000.0,31390.0,100.0,5.0,2937.0,1
31300,338000.0,31390.0,108.0,4.0,705.0,1
31301,216000.0,31390.0,129.0,6.0,80.0,1


## Save the clean data, separated House and Apartement (Appt)

In [58]:
df.to_csv("HousePrice_Tls_2021_2023.csv",index=False)